In [18]:
import pandas as pd
import numpy as np

import os
import shutil
import glob

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.pandas as ps

# Load Data

In [2]:
spark = SparkSession.builder \
    .appName("Read JSON") \
    .getOrCreate()

df_offers = spark.read.json("../data/raw/offers.json")
df_profile = spark.read.json("../data/raw/profile.json")
df_transactions = spark.read.json("../data/raw/transactions.json")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 21:41:42 WARN Utils: Your hostname, pedro-pc, resolves to a loopback address: 127.0.1.1; using 192.168.15.69 instead (on interface wlo1)
26/01/13 21:41:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/13 21:41:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_offers.pandas_api()

,channels,discount_value,duration,id,min_value,offer_type
0,"[email, mobile, social]",10,7.0,ae264e3637204a6fb9bb56bc8210ddfd,10,bogo
1,"[web, email, mobile, social]",10,5.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10,bogo
2,"[web, email, mobile]",0,4.0,3f207df678b143eea3cee63160fa8bed,0,informational
3,"[web, email, mobile]",5,7.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,bogo
4,"[web, email]",5,10.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,discount
5,"[web, email, mobile, social]",3,7.0,2298d6c36e964ae4a3e7e9706d1fb8c2,7,discount
6,"[web, email, mobile, social]",2,10.0,fafdcd668e3743c1bb461111dcafc2a4,10,discount
7,"[email, mobile, social]",0,3.0,5a8bc65990b245e5a138643cd4eb9837,0,informational
8,"[web, email, mobile, social]",5,5.0,f19421c1d4aa40978ebb69ca19b0e20d,5,bogo
9,"[web, email, mobile]",2,7.0,2906b810c7d4411798c6938adc9daaa5,10,discount


In [4]:
df_profile.pandas_api().head(20)

,age,credit_card_limit,gender,id,registered_on
0,118,NaN,None,68be06ca386d4c31939f3a4f0e3dd783,20170212
1,55,112000.0,F,0610b486422d4921ae7d2bf64640c50b,20170715
2,118,NaN,None,38fe809add3b4fcf9315a9694bb96ff5,20180712
3,75,100000.0,F,78afa995795e4d85b5d9ceeca43f5fef,20170509
4,118,NaN,None,a03223e636434f42ac4c3df47e8bac43,20170804
5,68,70000.0,M,e2127556f4f64592b11af22de27a7932,20180426
6,118,NaN,None,8ec6ce2a7e7949b1bf142def7d0e0586,20170925
7,118,NaN,None,68617ca6246f4fbc85e91a2a49552598,20171002
8,65,53000.0,M,389bc3fa690240e798340f5a15918d5c,20180209
9,118,NaN,None,8974fc5686fe429db53ddde067b88302,20161122


In [5]:
df_transactions.pandas_api().head(30)

,account_id,event,time_since_test_start,value
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0.0,"(None, 9b98b8c7a33c4b65b9aebfe6a799e6d9, None,..."
1,a03223e636434f42ac4c3df47e8bac43,offer received,0.0,"(None, 0b1e1539f2cc45b7b9fa7c272da2e1d7, None,..."
2,e2127556f4f64592b11af22de27a7932,offer received,0.0,"(None, 2906b810c7d4411798c6938adc9daaa5, None,..."
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0.0,"(None, fafdcd668e3743c1bb461111dcafc2a4, None,..."
4,68617ca6246f4fbc85e91a2a49552598,offer received,0.0,"(None, 4d5c57ea9a6940dd891ad53e9dbe8da0, None,..."
5,389bc3fa690240e798340f5a15918d5c,offer received,0.0,"(None, f19421c1d4aa40978ebb69ca19b0e20d, None,..."
6,c4863c7985cf408faee930f111475da3,offer received,0.0,"(None, 2298d6c36e964ae4a3e7e9706d1fb8c2, None,..."
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0.0,"(None, 3f207df678b143eea3cee63160fa8bed, None,..."
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,0.0,"(None, 0b1e1539f2cc45b7b9fa7c272da2e1d7, None,..."
9,31dda685af34476cad5bc968bdb01c53,offer received,0.0,"(None, 0b1e1539f2cc45b7b9fa7c272da2e1d7, None,..."


# Cleaning

In [6]:
df_offers = df_offers.pandas_api().rename(columns={'id': 'offer_id'})

## Profile

In [7]:
df_profile = df_profile.pandas_api().rename(columns={'id': 'account_id'})
df_profile.head(20)

,age,credit_card_limit,gender,account_id,registered_on
0,118,NaN,None,68be06ca386d4c31939f3a4f0e3dd783,20170212
1,55,112000.0,F,0610b486422d4921ae7d2bf64640c50b,20170715
2,118,NaN,None,38fe809add3b4fcf9315a9694bb96ff5,20180712
3,75,100000.0,F,78afa995795e4d85b5d9ceeca43f5fef,20170509
4,118,NaN,None,a03223e636434f42ac4c3df47e8bac43,20170804
5,68,70000.0,M,e2127556f4f64592b11af22de27a7932,20180426
6,118,NaN,None,8ec6ce2a7e7949b1bf142def7d0e0586,20170925
7,118,NaN,None,68617ca6246f4fbc85e91a2a49552598,20171002
8,65,53000.0,M,389bc3fa690240e798340f5a15918d5c,20180209
9,118,NaN,None,8974fc5686fe429db53ddde067b88302,20161122


In [8]:
df_profile['age'].hist()

In [9]:
df_profile = df_profile[df_profile['age'] < 98]
df_profile.head(20)

,age,credit_card_limit,gender,account_id,registered_on
1,55,112000.0,F,0610b486422d4921ae7d2bf64640c50b,20170715
3,75,100000.0,F,78afa995795e4d85b5d9ceeca43f5fef,20170509
5,68,70000.0,M,e2127556f4f64592b11af22de27a7932,20180426
8,65,53000.0,M,389bc3fa690240e798340f5a15918d5c,20180209
12,58,51000.0,M,2eeac8d8feae4a8cad5a6af0499a211d,20171111
13,61,57000.0,F,aa4862eba776480b8bb9c68455b8c2e1,20170911
14,26,46000.0,M,e12aeaf2d47d42479ea1c4ac3d8286c6,20140213
15,62,71000.0,F,31dda685af34476cad5bc968bdb01c53,20160211
16,49,52000.0,M,62cf5e10845442329191fc246e7bcea3,20141113
18,57,42000.0,M,6445de3b47274c759400cd68131d91b4,20171231


## Transactions

In [10]:
df_transactions = df_transactions.select("*", "value.*")
df_transactions = df_transactions.pandas_api()
df_transactions['offer_id'] = df_transactions.apply(
    lambda row: row['offer id'] if pd.notnull(row['offer id']) else row['offer_id'],
    axis=1
)
df_transactions = df_transactions.drop(columns=['offer id', 'value'])

/home/pedro/miniconda3/envs/myenvgpu/lib/python3.11/site-packages/pyspark/pandas/utils.py:1038: PandasAPIOnSparkAdviceWarning:

If the type hints is not specified for `apply`, it is expensive to infer the data type internally.



In [11]:
df_transactions.head(20)

26/01/13 21:42:05 WARN AttachDistributedSequenceExec: clean up cached RDD(73) in AttachDistributedSequenceExec(301)
26/01/13 21:42:09 WARN AttachDistributedSequenceExec: clean up cached RDD(78) in AttachDistributedSequenceExec(340)


,account_id,event,time_since_test_start,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0.0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
6,c4863c7985cf408faee930f111475da3,offer received,0.0,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0.0,NaN,3f207df678b143eea3cee63160fa8bed,NaN
9,31dda685af34476cad5bc968bdb01c53,offer received,0.0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
17,65aba5c617294649aeb624da249e1ee5,offer received,0.0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
19,1e9420836d554513ab90eba98552d0a9,offer received,0.0,NaN,ae264e3637204a6fb9bb56bc8210ddfd,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0.0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0.0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0.0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
5,389bc3fa690240e798340f5a15918d5c,offer received,0.0,NaN,f19421c1d4aa40978ebb69ca19b0e20d,NaN


# Merge

In [12]:
loss_offer_join = len(df_transactions[~df_transactions['offer_id'].isin(df_offers['offer_id'].to_numpy())])/len(df_transactions)
df_final = df_offers.merge(df_transactions, on='offer_id', how='inner')

print('Perda de transações ao juntar com ofertas: {:.2f}%'.format(loss_offer_join*100))

/home/pedro/miniconda3/envs/myenvgpu/lib/python3.11/site-packages/pyspark/pandas/utils.py:1038: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.

26/01/13 21:42:18 WARN AttachDistributedSequenceExec: clean up cached RDD(103) in AttachDistributedSequenceExec(485)
26/01/13 21:42:21 WARN AttachDistributedSequenceExec: clean up cached RDD(108) in AttachDistributedSequenceExec(560)
26/01/13 21:42:22 WARN AttachDistributedSequenceExec: clean up cached RDD(130) in AttachDistributedSequenceExec(754)
26/01/13 21:42:24 WARN AttachDistributedSequenceExec: clean up cached RDD(135) in AttachDistributedSequenceExec(825)


Perda de transações ao juntar com ofertas: 45.33%


In [13]:
loss_profile_join = len(df_final.merge(df_profile, on='account_id', how='inner'))/len(df_final)
df_final = df_final.merge(df_profile, on='account_id', how='inner')

print('Perda de transações ao juntar com perfis: {:.2f}%'.format((1 - loss_profile_join)*100))

26/01/13 21:42:30 WARN AttachDistributedSequenceExec: clean up cached RDD(157) in AttachDistributedSequenceExec(1056)
26/01/13 21:42:33 WARN AttachDistributedSequenceExec: clean up cached RDD(168) in AttachDistributedSequenceExec(1182)
26/01/13 21:42:33 WARN AttachDistributedSequenceExec: clean up cached RDD(192) in AttachDistributedSequenceExec(1610)


Perda de transações ao juntar com perfis: 11.37%


26/01/13 21:42:36 WARN AttachDistributedSequenceExec: clean up cached RDD(200) in AttachDistributedSequenceExec(1798)


In [24]:
tmp_path = "../data/tmp_processed/"
final_path = "../data/processed/processed_data.json"

df_final.to_spark().coalesce(1).write.mode('overwrite').json(tmp_path)

part_file = glob.glob(os.path.join(tmp_path, "part-*.json"))[0]

shutil.move(part_file, final_path)
shutil.rmtree(tmp_path)

/home/pedro/miniconda3/envs/myenvgpu/lib/python3.11/site-packages/pyspark/pandas/utils.py:1038: PandasAPIOnSparkAdviceWarning:

If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.

26/01/13 22:08:19 WARN AttachDistributedSequenceExec: clean up cached RDD(512) in AttachDistributedSequenceExec(6569)
26/01/13 22:08:21 WARN AttachDistributedSequenceExec: clean up cached RDD(515) in AttachDistributedSequenceExec(6609)
